# (실습준비) 데이터 불러오기부터

* 17년도부터 21년도 까지의 데이터를 train set, 22년도의 6개월치 데이터를 test set 으로 설정합니다.

In [1]:
import pandas as pd

# 학습 데이터 로딩
df_total = pd.read_csv('./data/df_total.csv')
df_test = pd.read_csv('./data/df_test.csv')

In [2]:
df_total = pd.concat([df_total, df_test], axis=0)

#### [실습문제1] 열 추가 
* 기존 '기준일ID'열에서 날짜 데이터를 추출하여 새로운 열로 추가 합니다.
    - pd.to_datetime() 함수를 사용해 '기준일ID'열을 datetime 형식으로 변경하겠습니다.
    - year, month, day 열로 기존 데이터 프레임에 연월일 정보를 저장합니다.
    - 데이터 프레임 열에 대해 날짜 정보를 얻을 때에는 **dt.year**, **dt.month**, **dt.day** 속성을 사용합니다. 

In [3]:
df_total['기준일ID'] = pd.to_datetime(df_total['기준일ID'])

df_total['year']=df_total['기준일ID'].dt.year
df_total['month']=df_total['기준일ID'].dt.month
df_total['day']=df_total['기준일ID'].dt.day

#### [실습문제2] 불필요한 열 제거 및 열 이름 변경
* df_total과 df_test에서 '총생활인구수'외에 불필요한 열을 제거 하도록 하겠습니다.
* 시간 데이터를 남기고, '시간대구분' column 을 'hour'로 rename 하도록 하겠습니다. 

In [4]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_total.rename(columns={'시간대구분': 'hour'}, inplace=True)
df_total = df_total.loc[:, ['year', 'month', 'day', 'hour', '총생활인구수']].copy()

#### [실습문제3] 시프트 & 롤링
* shift()를 활용하여 이동할 행 수를 지정합니다.
* rolling()을 활용하여 추가할 이동평균 수를 지정합니다.

In [5]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_total['총생활인구수_D+7'] = df_total['총생활인구수'].shift(-168) # 7일 후 동일 시간대의 총 생활인구수
df_total['총생활인구수_M+1'] = df_total['총생활인구수'].shift(-720) # 1개월(30일) 후 동일 시간대의 총 생활인구수
df_total['총생활인구수_M+2'] = df_total['총생활인구수'].shift(-1416) # 2개월 후 동일 시간대의 총 생활인구수
df_total['총생활인구수_Y+1'] = df_total['총생활인구수'].shift(-8760) # 1년 후 동일 시간대의 총 생활인구수

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# df_total['총생활인구수_MA_168'] = df_total['총생활인구수'].rolling(168).mean() # 기준일 포함 최근 7일간 이동 평균

#### [실습문제4] 결측치 처리 및 불필요한 열 제거
* 롤링 & 시프트 과정에서 발생한 결측치를 처리하도록 하겠습니다.
* 불필요한 열을 추가로 제거 하겠습니다.(시간 관련 행들)

In [6]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
cols = ['year', 'month', 'day', 'hour']

df_total.drop(cols, axis=1, inplace=True)

In [7]:
df_total.head()

,총생활인구수,총생활인구수_D+7,총생활인구수_M+1,총생활인구수_M+2,총생활인구수_Y+1
0,31535.2200,30716.3268,30539.0226,31209.1023,36514.0589
1,31188.9174,30673.4813,30445.4356,30334.7708,37128.7367
2,31240.4974,30535.9573,30571.1750,30558.3055,37799.9458
3,31442.4314,30884.0372,30661.7845,30789.1289,38004.9325
4,31922.7751,31353.9409,31212.8186,31420.6076,38744.0673


In [8]:
dataset_1 = df_total.loc[:, ['총생활인구수', '총생활인구수_D+7']].copy() # 7일 후의 생활인구수 예측
dataset_2 = df_total.loc[:, ['총생활인구수', '총생활인구수_M+1']].copy() # 1달 후의 생활인구수 예측
dataset_3 = df_total.loc[:, ['총생활인구수', '총생활인구수_M+2']].copy() # 2달 후의 생활인구수 예측
dataset_4 = df_total.loc[:, ['총생활인구수', '총생활인구수_Y+1']].copy() # 1년 후의 생활인구수 예측

In [9]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
dataset_1.dropna(axis=0, inplace=True)
dataset_2.dropna(axis=0, inplace=True)
dataset_3.dropna(axis=0, inplace=True)
dataset_4.dropna(axis=0, inplace=True)

#### [실습문제5] train, test 데이터 나누기
* df_total과 df_test를 각각 train_x, train_y, test_x, test_y 로 나누어 주세요.
* 인덱스를 리셋 해주세요.

In [10]:
from sklearn.model_selection import train_test_split

In [20]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
x_1 = dataset_1['총생활인구수'].values
y_1 = dataset_1['총생활인구수_D+7'].values

train_x_1, test_x_1, train_y_1, test_y_1 = train_test_split(x_1, y_1, test_size=.2)

In [21]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
x_2 = dataset_2['총생활인구수'].values
y_2 = dataset_2['총생활인구수_M+1'].values

train_x_2, test_x_2, train_y_2, test_y_2 = train_test_split(x_2, y_2, test_size=.2)

In [22]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
x_3 = dataset_3['총생활인구수'].values
y_3 = dataset_3['총생활인구수_M+2'].values

train_x_3, test_x_3, train_y_3, test_y_3 = train_test_split(x_3, y_3, test_size=.2)

In [23]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
x_4 = dataset_4['총생활인구수'].values
y_4 = dataset_4['총생활인구수_Y+1'].values

train_x_4, test_x_4, train_y_4, test_y_4 = train_test_split(x_4, y_4, test_size=.2)

In [24]:
train_x = [train_x_1, train_x_2, train_x_3, train_x_4]
train_y = [train_y_1, train_y_2, train_y_3, train_y_4]
test_x = [test_x_1, test_x_2, test_x_3, test_x_4]
test_y = [test_y_1,test_y_2, test_y_3, test_y_4]

----

### 가. LinearRegression 부터 시작해봅시다.

#### [실습문제1] 머신러닝_1
* Train과 Test로 나눈 데이터를 기준으로 LinearRegression 모델링을 진행하고 평가를 해주세요.
* 성능지표 : RMSE, R-squared Score

In [25]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import *

In [33]:
# 아래에 실습코드를 작성하세요.
for i, (tx, ty, ttx, tty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    lr = LinearRegression()
    lr.fit(tx.reshape(-1, 1), ty)
    
    pred_lr = lr.predict(ttx.reshape(-1, 1))

    if i==0:
        print('--- 7일 후의 생활인구수 예측 ---')
    elif i==1:
        print('--- 1달 후의 생활인구수 예측 ---')
    elif i==2:
        print('--- 2달 후의 생활인구수 예측 ---')
    else:
        print('--- 1년 후의 생활인구수 예측 ---')
        
    print('RMSE: ', mean_squared_error(tty, pred_lr, squared=False))
    print('R-squared Score: ', r2_score(tty, pred_lr))
    print()

--- 7일 후의 생활인구수 예측 ---
RMSE:  1776.69081227391
R-squared Score:  0.8256857440269225

--- 1달 후의 생활인구수 예측 ---
RMSE:  2938.0656059078483
R-squared Score:  0.5312835971594729

--- 2달 후의 생활인구수 예측 ---
RMSE:  3194.7571725818657
R-squared Score:  0.44337793328743624

--- 1년 후의 생활인구수 예측 ---
RMSE:  3575.1304944341878
R-squared Score:  0.29243453783871876



#### [실습문제2] 머신러닝_2
* Train과 Test로 나눈 데이터를 기준으로 렌덤포레스트로 모델을 학습을 진행하고 평가를 해주세요.
* 성능지표 : RMSE, R-squared Score

In [34]:
from sklearn.ensemble import RandomForestRegressor

In [35]:
# 아래에 실습코드를 작성하세요.
for i, (tx, ty, ttx, tty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    rf = RandomForestRegressor()
    rf.fit(tx.reshape(-1, 1), ty)
    
    pred_rf = rf.predict(ttx.reshape(-1, 1))

    if i==0:
        print('--- 7일 후의 생활인구수 예측 ---')
    elif i==1:
        print('--- 1달 후의 생활인구수 예측 ---')
    elif i==2:
        print('--- 2달 후의 생활인구수 예측 ---')
    else:
        print('--- 1년 후의 생활인구수 예측 ---')
        
    print('RMSE: ', mean_squared_error(tty, pred_rf, squared=False))
    print('R-squared Score: ', r2_score(tty, pred_rf))
    print()

--- 7일 후의 생활인구수 예측 ---
RMSE:  2086.887958968078
R-squared Score:  0.7595042200749691

--- 1달 후의 생활인구수 예측 ---
RMSE:  3494.5976335812275
R-squared Score:  0.336896178767786

--- 2달 후의 생활인구수 예측 ---
RMSE:  3829.354844942175
R-squared Score:  0.20028372298524977

--- 1년 후의 생활인구수 예측 ---
RMSE:  4245.643265085746
R-squared Score:  0.0021395044513651262



#### [실습문제3] 머신러닝_3
* Train과 Test로 나눈 데이터를 기준으로 그라디언트부스팅으로 모델을 학습을 진행하고 평가를 해주세요.
* 성능지표 : RMSE, R-squared Score

In [36]:
from xgboost import XGBRegressor

In [37]:
# 아래에 실습코드를 작성하세요.
for i, (tx, ty, ttx, tty) in enumerate(zip(train_x, train_y, test_x, test_y)):
    xgb = XGBRegressor()
    xgb.fit(tx.reshape(-1, 1), ty)
    
    pred_xgb = xgb.predict(ttx.reshape(-1, 1))

    if i==0:
        print('--- 7일 후의 생활인구수 예측 ---')
    elif i==1:
        print('--- 1달 후의 생활인구수 예측 ---')
    elif i==2:
        print('--- 2달 후의 생활인구수 예측 ---')
    else:
        print('--- 1년 후의 생활인구수 예측 ---')
        
    print('RMSE: ', mean_squared_error(tty, pred_xgb, squared=False))
    print('R-squared Score: ', r2_score(tty, pred_xgb))
    print()

--- 7일 후의 생활인구수 예측 ---
RMSE:  1763.9992776997028
R-squared Score:  0.8281672266948278

--- 1달 후의 생활인구수 예측 ---
RMSE:  2921.1679095342
R-squared Score:  0.5366595504009777

--- 2달 후의 생활인구수 예측 ---
RMSE:  3189.933334670412
R-squared Score:  0.44505757738538154

--- 1년 후의 생활인구수 예측 ---
RMSE:  3547.7526173504066
R-squared Score:  0.3032299298115201



----